Notebook Logistic Regression Case
Oefening Data Scientist 
Geert Vandezande

Doel:
- Supervised Learning toepassen
- EDA uitvoeren op een dataset
- Logistic Regression toepassen op de data voor classificatie
- andere vormen van classificatie toepassen

Dataset: 
- More info: see kaggle https://www.kaggle.com/datasets/arifmia/heart-attack-risk-dataset


Volgorde van activiteiten in deze notebook: (cfr Datacamp "preparing data for modelling)
- data inlezen
- data bekijken, visueel en numerisch
- missing data oplossen 
- incorrect types controleren
- numerische waarde standardizeren
- categorische varaiabelen processen
- feature engineering
- select features for modelling
- eenvoudige logistic regression
- eenvoudige logistic regression en unbalance van de features corrigeren met SMOTE- 
- modeling met Logistic Regression, Decision Tree Regression en Random Forrest Regression
- modeling met Hyperparameter tuning met GridSearchCV voor één model


Aantal testen uitgevoerd, met en zonder SMOTE, met en zonder HyperParameter tuning maar de resultaten veranderen niet zo veel
Ook logistic regression met class_weight='balanced' maar opzich weinig verschil in de resultaten



In [ ]:
# import van de diverse modules
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.datasets import make_classification
from sklearn.ensemble import IsolationForest
from collections import Counter
from sklearn.base import TransformerMixin, BaseEstimator
from imblearn.under_sampling import RandomUnderSampler

# Machine learning algorithm
from statsmodels.formula.api import ols
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB

# Evaluation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# system utils
import warnings
from pathlib import Path
import datetime
from colorama import Fore, Back, Style
import sys
import os
import chardet
from summarytools import dfSummary
from tabulate import tabulate


plot_graphs = True

Extra code snippits die doorheen de notebook gebruikt worden:

save_fig: na generatie van een image kan de image naar file geschreven worden in de images/.. directory. Geef steeds een zinvolle naam

read_JSON: om eenvoudig een JSON in te lezen

log_info:
- logging functie om doorheen de notebooks de status naar file te kunnen schrijven. 
- de logstatements worden tijdens de uitvoering van de code bewaard in een list. Die kan tussentijds naar het scherm geprint worden of naar een file
- log_info_write_to_file: schrijf de loginformatie naar file 
- log_info_print_on_screen: print alle loginfo naar het scherm

In [ ]:
# enkele extra code snippets gebruikt doorheen de oefening

# to plot or not to plot - zet op True om de plots te zien, zet op False om de plots niet te zien bij een Run ALL
plot_graphs = True


# schrijf een visual naar file

IMAGES_PATH = Path() / "images" 
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Lezen van de JSON-file
def read_JSON(file_path_read):
    with open(file_path_read, 'r') as file:
        files_from_json = json.load(file)
    return files_from_json

# functies om te loggen naar file
log_info_lijst = []

log_filenaam = "LogReg_continue.log"
if os.path.exists(log_filenaam):
    os.remove(log_filenaam)

def log(log_code="INFO", boodschap="euh geen boodschap????"):
    global log_info_lijst
    now = datetime.datetime.now()
    formatted_date = now.strftime("%d/%m/%Y %H:%M:%S")
    log_message = f"{Style.RESET_ALL}{formatted_date} : {log_code} : {boodschap}"
    log_info_lijst.append(log_message)
    with open(log_filenaam, 'a') as file:  # Open the file in append mode
        file.write(boodschap + '\n')  # Voeg een nieuwe regel toe na elke string
    print(log_message)
    return

def log_info(boodschap):
    log("Info",boodschap)
    def log_info_write_to_file(filename):
        with open(filename, 'a') as file:  # Open the file in append mode
            for string in log_info_lijst:
                file.write(string + '\n')  # Voeg een nieuwe regel toe na elke string
        return

def log_info_write_to_file(filename):
    with open(filename, 'w') as file:
        for string in log_info_lijst:
            file.write(string + '\n')  # Voeg een nieuwe regel toe na elke string
    return

def log_info_print_on_screen():
    for boodschap in log_info_lijst:
        print(boodschap)    
    return

In [ ]:
def plot_histogram(dataframe, column, bins=20, color='blue', title='', xlabel='', ylabel='Frequentie', filenaam = "Histogram"):
    """
    Deze functie plot een histogram van een gespecificeerde kolom uit een pandas DataFrame.
    
    Parameters:
        dataframe (pd.DataFrame): Het DataFrame dat de data bevat.
        column (str): De naam van de kolom waarvan een histogram moet worden geplot.
        bins (int): Het aantal bins (groepen) voor het histogram.
        color (str): De kleur van de histogram bars.
        title (str): De titel van de plot.
        xlabel (str): De label voor de x-as.
        ylabel (str): De label voor de y-as.
    """
    # Controleer of de kolom bestaat in het DataFrame
    if column not in dataframe.columns:
        print(f"Kolom '{column}' niet gevonden in het DataFrame.")
        return

    # Plot het histogram
    plt.hist(dataframe[column], bins=bins, color=color, alpha=0.7)
    plt.title(title if title else f'Histogram van {column}')
    plt.xlabel(xlabel if xlabel else column)
    plt.ylabel(ylabel)
    if plot_graphs:
        save_fig(filenaam)
        plt.show()

Hulpfuncties

bereken_percentage_outliers: via Isolation forest wordt het percentage van de outliers berekend.
- df : dataframe
- columns_to_use: list van koloms die gebruikt worden om de outliers te berekenen
- functie geeft een percentage terug van het aaantal outliers op het totaal aantal observaties


cap_values: vervang outliers door hun lower of upperpercentieel waarde: de whiskers worden berekend door van de lower_percentieel waarde een waarde af te trekken gelijk aan 1,5 * IQR (interquartile range), voor upper_percentieel waarde wordt de 1,5 * IQR bijgeteld
- df: dataframe
- columns_to_use
- lower_percentieel (default = 25)
- upper_percentieel (default = 75)


In [ ]:
# functie om het percentage outliers te berkenen voor een set van kolommen in een dataframe
def bereken_percentage_aantal_outliers(df , columns_to_use):
    # Initialiseren van het Isolation Forest model
    iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)

    # Fit het model
    iso_forest.fit(df[columns_to_use])
    # Voorspellingen
    # Het geeft -1 voor outliers en 1 voor inliers
    labels = iso_forest.predict(df[columns_to_use])
    # Toevoegen van de labels aan het DataFrame om outliers te identificeren
    df_intern = df.copy()
    df_intern['outlier'] = labels
    outliers = df_intern[df_intern['outlier'] == -1]
    aantal_outliers = df_intern['outlier'].value_counts()
    print(aantal_outliers)
    percentage_aantal_outliers = (len(outliers) / len(df_intern)) * 100

    return percentage_aantal_outliers


# functie om outliers in een kolom te cappen op een percentiel waarde
def cap_values(df_input, column, lower_percentile=25, upper_percentile=75):
    # voeg code toe om beter de outliers te verwijderen
    log("Info", f"Capping values voor kolom {column} naar lower percentiel {lower_percentile} - upper percentiel {upper_percentile}")
    q1, q3 = np.percentile(df_input[column], [lower_percentile, upper_percentile])  # Calculate the 25th (Q1) and 75th (Q3) percentiles
    iqr = q3 - q1  # Calculate the interquartile range (IQR)
    lower_bound = q1 - 1.5 * iqr  # Calculate lower whisker (Q1 - 1.5 * IQR)
    upper_bound = q3 + 1.5 * iqr  # Calculate upper whisker (Q3 + 1.5 * IQR)

    # lower_bound = df[column].quantile(lower_percentile)
    # upper_bound = df[column].quantile(upper_percentile)
    
    # Waarden cappen met behulp van de numpy.where functie
    df_output = df_input.copy()
    df_output[column] = np.where(df_input[column] < lower_bound, lower_bound, df_input[column])
    df_output[column] = np.where(df_input[column] > upper_bound, upper_bound, df_input[column])    
    return df_output


In [ ]:
# data inlezen van de file
heart_attack_risk_dataset_filename = 'data/heart_attack_risk_dataset.csv'
df = pd.read_csv(heart_attack_risk_dataset_filename)
log_info(f"File ingelezen: {heart_attack_risk_dataset_filename}")
df_raw = df.copy() # hou het ruwe dataframe bij

# lijst met features opgedeeld in numerisch en categorisch
df_col = df.columns
print(df_col)

# bepaal eerst de opdeling van numerische en categorische waarden op basis van de kolommen
# 3 opties

# optie 1: op basis van het datatype in de kolome
# df_num_col = df.select_dtypes(include='number').columns
# df_cat_col = df.select_dtypes(include='object').columns


# optie 2: op basis van de voorkomens van de waarden, indien <=4 is het een categorische 
# minder goed
# df_cat_col = [i for i in df.columns if df[i].nunique() <= 4]
# df_num_col= [i for i in df.columns if i not in df_cat_col]

# optie 3: en dit is de manuele manier om de lijst van kolommen voor numerische en categorische waarden samen te stellen
# check in via info() en describe() 
# we gaan dit verder gebruiken
df_num_col = ['Age', 'BMI', 'Cholesterol_Level', 'Resting_BP', 'Heart_Rate', 'Max_Heart_Rate_Achieved']
df_cat_col = list(set(df_col) - set(df_num_col))
df_label_col = ['Heart_Attack_Risk'] # dit is de te voorspellen waarde
df_cat_nom_col = ['Gender', 'Physical_Activity_Level', 'Stress_Level', 'Chest_Pain_Type', 'Thalassemia', 'ECG_Results', 'Heart_Attack_Risk']
df_cat_ord_col = list(set(df_cat_col) - set(df_cat_nom_col))

df_num = df[df_num_col]
df_cat = df[df_cat_col]
df_label = df[df_label_col]
df_cat_nom = df[df_cat_nom_col]
df_cat_ord = df[df_cat_ord_col]

In [ ]:
# eerste controles van alle waarden: 
# zijn er nulwaarden?
# zitten er geen duplicates tussen?

df.head()

# Maak een samenvatting 
df_summary = df.describe().transpose()
# Zet DataFrame om naar een mooie teksttabel
summary_str = tabulate(df_summary, headers='keys', tablefmt='psql')
# Print de string
log_info(f"\nDescribe van de dataset")
log_info(summary_str)

# check op nulwaarden:
aantal_nul_waarden = df.isnull().sum()
log_info(f"\nCheck op nulwaarden: \n{aantal_nul_waarden}")
# ok, geen nulwaarden

aantal_duplicated_waarden = df.duplicated().sum()
log_info(f"\nCheck op duplicates: \n{aantal_duplicated_waarden}")
# ok, geen duplicates

In [ ]:
# We nemen eerst een snelle blik op de variabelen via een histogram plot van alle features
# We maken een algemene plot van de variabelen

if plot_graphs:
    num_columns = 3
    num_plots = len(df.columns)
    num_rows = (num_plots + num_columns - 1) // num_columns
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, num_rows * 5))
    axes = axes.flatten()
    for i, column in enumerate(df.columns):
        df[column].hist(bins=50, ax=axes[i])
        axes[i].set_title(column)
    # Hide any unused subplots
    for i in range(num_plots, len(axes)):
        fig.delaxes(axes[i])
    plt.tight_layout()
    save_fig("algemeen_overzicht_features")
    plt.show()

In [ ]:
# analyze van de features, eerst de numerische features
# eerst een boxplot, voor de numerische waarde

# outliers berekenen maar hier doen we voorlopig niets mee
# dit lijkt hoog te zijn maar we gaan dit niet gebruiken
aantal_outliers = bereken_percentage_aantal_outliers(df,['Age'])
print(aantal_outliers)

# eerste een box-plot van de numerische variabelen
if plot_graphs:
    fig = plt.figure(figsize=(30,10))
    sns.boxplot(df_num)
    save_fig("Numerische features boxplot van de features")
    plt.show()
# Dit lijkt een normale verdeling. Er is hier geen log-verfijning nodig
# Er zijn ook geen outliers

# We maken nog een apart histogram van de numersiche variabelen
plt.figure(figsize=(20,14))
for i, col in enumerate(df_num_col,1):
    plt.subplot(5,3,i)
    sns.histplot(df[col], kde=True, palette='skyblue')
    plt.title(f"Distribution of {col}")
if plot_graphs:
    plt.tight_layout()
    save_fig("Numerische features histogram van de features")
    plt.show()
# Alles lijkt ook hier normaal



In [ ]:
# check correlatie tussen de diverse features

data_num = pd.get_dummies(df, columns=['Gender','Physical_Activity_Level', 'Stress_Level','Chest_Pain_Type', 'Thalassemia', 'ECG_Results', 'Heart_Attack_Risk'], drop_first=True)

plt.figure(figsize=(25,25))
sns.heatmap(data_num.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

# weinig correlatie tussen de features
# we kunnen de features dus behouden

In [ ]:
# analyze van de categorische features
print(df_cat.head())
print(df_cat.describe())

In [ ]:
# maak per categorsiche variabel een bar_plot van de voorkomens van de waarden
def bar_labels(axes, rotation= 0, location="edge"):
    for container in axes.containers:
        axes.bar_label(container, rotation=rotation, label_type=location)
    axes.set_ylabel("")
    axes.set_xlabel("")
    axes.set_yticklabels(())

# plot de categorische waarde af met het voorkomen van de waarde in de dataset
# 14 plots, opgedeeld in 2 rijen met 7 plots

if plot_graphs:
    index = 0
    for r in [5,4,4]:
        fig, axes = plt.subplots(ncols=r, figsize=(15, 6))
        for i in range(r):
            df[df_cat_col[index]].value_counts().plot(kind="bar", ax=axes[i])
            bar_labels(axes[i])
            axes[i].set_title(df_cat_col[index].replace('_', ' '))            
            index+=1      
        plt.tight_layout()   
        save_fig(f"Categorische features value count van de feature deel{r}")
        plt.show()


In [ ]:
# visuele voorstellen van de unbalance van de categorische variabelen
# Creëer een figuur en een set van subplots
num_columns = 3
num_plots = len(df_cat.columns)
num_rows = (num_plots + num_columns - 1) // num_columns
fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, num_rows * 5))

# Flatten het axes object voor eenvoudiger iteratie
axes = axes.flatten()

# Loop over alle kolommen en creëer een pie chart voor elke kolom
for i, column in enumerate(df_cat.columns):
    # Haal de waarde tellingen op voor de huidige kolom
    counts = df_cat[column].value_counts()
    
    # Maak een pie chart op de i-th positie in axes
    axes[i].pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90)
    axes[i].set_title(column)
    axes[i].set_ylabel('')  # Verwijder de y-label voor netheid

# Verberg eventuele extra subplots als het aantal kolommen minder is dan het aantal subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

# Toon de plot
plt.tight_layout()
save_fig(f"Categorische features unbalance view")
plt.show()



In [ ]:
# SMOTE toepassen
# we gaan dit later mee in de pipelining integreren om te kijken wat het effect van SMOTE kan zijn
# sample test

# Voorbeeld dataset genereren
X, y = make_classification(n_classes=2, class_sep=2, weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0,
                           n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)

# Data in een DataFrame plaatsen voor visualisatie
df_smote = pd.DataFrame(X)
df_smote['target'] = y

# Toon de klasse distributie voor SMOTE
print('Originele dataset shape %s' % Counter(y))

# Pas SMOTE toe
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

# Toon de nieuwe klasse distributie
print('Resampled dataset shape %s' % Counter(y_res))

# Toon de originele klasse distributie
print('Originele dataset shape %s' % Counter(y))

# Configureer de RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_res_under, y_res_under = rus.fit_resample(X, y)

# Toon de nieuwe klasse distributie
print('Dataset shape na undersampling %s' % Counter(y_res_under))


In [ ]:
# en nu gaan we op basis van de target value ("Heart risk") groeperen en kijken wat de impact is van de diverse
# categorische variabelen

index = 0
grouped = df.groupby(df_label_col)
df_group_cols = list(set(df_cat_col) - set(df_label_col))
# plot 13 grafieken, 
if plot_graphs:
    for j in [5, 4, 4]:
        fig, axes = plt.subplots(ncols=j, figsize=(15, 6))
        for i in range(j):
            grouped[df_group_cols[index]].value_counts().unstack().plot(kind="bar", stacked=True, ax=axes[i])
            bar_labels(axes[i], 0, "center")
            axes[i].set_title(df_group_cols[index].replace('_', ' '))
            index+=1
        plt.tight_layout()
        save_fig(f"Categorische features per value van de target value deel {r}")
        plt.show()

Hier beginnen we met de modellen

Eerst een eenvoudige logistic regression, met stratefy op de target value
Eerst zonder SMOTE, dan met parameter class_weight = 'balanced" en dan eens met SMOTE om te zien wat het effect is van het aanpakken van de unbalanced data

In [ ]:
# Eenvoudige Logistic regression met pipeline opzet 
# met stratefy op de target value
# en zonder

log_info(f"Eenvoudige logistic regression met stratify op de target value zonder SMOTE")
log_info("**************************************************************")

df = df_raw.copy()

# Separate features (X) and target variable (y)
X = df.drop(columns=['Heart_Attack_Risk'])
y = df['Heart_Attack_Risk']

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Define categorical and numerical column names
df_col = X.columns  
df_num_col = ['Age', 'BMI', 'Cholesterol_Level', 'Resting_BP', 'Heart_Rate', 'Max_Heart_Rate_Achieved']
df_cat_col = list(set(df_col) - set(df_num_col))

num_pipeline = Pipeline([   
    ("standardize_numerical", MinMaxScaler(feature_range=(-1, 1)))
    # ("standardize_numerical", StandardScaler())
])

cat_pipeline = Pipeline(steps=[   
    ('encode_categorical', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer([
    ("numerical", num_pipeline, df_num_col),
    ("categorical", cat_pipeline, df_cat_col)],
     remainder='passthrough')

# Create the pipeline with preprocessing and logistic regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),       
    ('regressor', LogisticRegression())
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Making predictions on the test set
y_pred = pipeline.predict(X_test)

y_pred_counter = Counter(y_pred)
y_test_counter = Counter(y_test)
log_info(f"Voorkomens in y_pred: {y_pred_counter}")
log_info(f"Voorkomens in y_test: {y_test_counter}")

acc_score = accuracy_score(y_pred, y_test)*100
class_report = classification_report(y_pred, y_test)
conf_matrix = confusion_matrix(y_pred, y_test)

log_info(f"\naccuracy_score: {acc_score}")
log_info(f"classification_report: {class_report}")
summary_conf_matrix = tabulate(conf_matrix, headers='keys', tablefmt='psql')
log_info(f"Confusion matrix: {summary_conf_matrix}")

cm = confusion_matrix(y_test, y_pred, labels=pipeline.classes_)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=pipeline.classes_)
disp.plot()
save_fig("Confusion matrix - example logistic regression zonder SMOTE")


In [ ]:
# Eenvoudige Logistic regression met pipeline opzet 
# met stratefy op de target value
# en nu eens met de parameter class_weight='balanced' voor de logistic regression

log_info(f"Eenvoudige logistic regression met stratify op de target value en class weight = balanced")
log_info("******************************************************************************************")

df = df_raw.copy()

# Separate features (X) and target variable (y)
X = df.drop(columns=['Heart_Attack_Risk'])
y = df['Heart_Attack_Risk']

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Define categorical and numerical column names
df_col = X.columns  
df_num_col = ['Age', 'BMI', 'Cholesterol_Level', 'Resting_BP', 'Heart_Rate', 'Max_Heart_Rate_Achieved']
df_cat_col = list(set(df_col) - set(df_num_col))

num_pipeline = Pipeline([   
    ("standardize_numerical", MinMaxScaler(feature_range=(-1, 1)))
    # ("standardize_numerical", StandardScaler())
])

cat_pipeline = Pipeline(steps=[   
    ('encode_categorical', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer([
    ("numerical", num_pipeline, df_num_col),
    ("categorical", cat_pipeline, df_cat_col)],
     remainder='passthrough')

# Create the pipeline with preprocessing and logistic regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),       
    ('regressor', LogisticRegression(class_weight='balanced'))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Making predictions on the test set
y_pred = pipeline.predict(X_test)

y_pred_counter = Counter(y_pred)
y_test_counter = Counter(y_test)
log_info(f"Voorkomens in y_pred: {y_pred_counter}")
log_info(f"Voorkomens in y_test: {y_test_counter}")

acc_score = accuracy_score(y_pred, y_test)*100
class_report = classification_report(y_pred, y_test)
conf_matrix = confusion_matrix(y_pred, y_test)

log_info(f"\naccuracy_score: {acc_score}")
log_info(f"classification_report: {class_report}")
summary_conf_matrix = tabulate(conf_matrix, headers='keys', tablefmt='psql')
log_info(f"Confusion matrix: {summary_conf_matrix}")

cm = confusion_matrix(y_test, y_pred, labels=pipeline.classes_)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=pipeline.classes_)
disp.plot()
save_fig("Confusion matrix - example logistic regression met class weight = balanced")


Eenvoudige logistic regression met pipeline opzet en met SMOTE
Om te checken of SMOTE bijdraagt tot een beter resultaat

Maar dat is niet. Voor de vervolg regressie passen we geen SMOTE meer toe

In [ ]:
# Eenvoudige Logistic regression met pipeline opzet en met SMOTE
# met stratefy op de target value

log_info(f"Eenvoudige logistic regression met SMOTE en met stratify op de target value")
log_info("********************************************************************")

df = df_raw.copy()

# Separate features (X) and target variable (y)
X = df.drop(columns=['Heart_Attack_Risk'])
y = df['Heart_Attack_Risk']

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Define categorical and numerical column names
df_col = X.columns  
df_num_col = ['Age', 'BMI', 'Cholesterol_Level', 'Resting_BP', 'Heart_Rate', 'Max_Heart_Rate_Achieved']
df_cat_col = list(set(df_col) - set(df_num_col))

num_pipeline = Pipeline([   
    ("standardize_numerical", MinMaxScaler(feature_range=(-1, 1)))
    # ("standardize_numerical", StandardScaler())
])

cat_pipeline = Pipeline(steps=[   
    ('encode_categorical', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer([
    ("numerical", num_pipeline, df_num_col),
    ("categorical", cat_pipeline, df_cat_col)],
     remainder='passthrough')

# Create the pipeline with preprocessing and logistic regression
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('SMOTE', SMOTE(random_state=42)),      
    ('regressor', LogisticRegression(class_weight='balanced'))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Making predictions on the test set
y_pred = pipeline.predict(X_test)

y_pred_counter = Counter(y_pred)
y_test_counter = Counter(y_test)
log_info(f"Voorkomens in y_pred: {y_pred_counter}")
log_info(f"Voorkomens in y_test: {y_test_counter}")

acc_score = accuracy_score(y_test, y_pred)*100
class_report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

log_info(f"\naccuracy_score: {acc_score}")
log_info(f"classification_report: {class_report}")
summary_conf_matrix = tabulate(conf_matrix, headers='keys', tablefmt='psql')
log_info(f"Confusion matrix: {summary_conf_matrix}")

cm = confusion_matrix(y_test, y_pred, labels=pipeline.classes_)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=pipeline.classes_)
disp.plot()
save_fig("Confusion matrix - example logistic regression met SMOTE")



Nu gaan we voor een aantal classificatie modellen met pipeling
Zonder SMOTE omdat dit niet echt bijdraagt tot het resultaat

In [ ]:
# en nu gaan we voor een aantal classificatie modellen dit uitvoeren
# en checken wat het beste model is tot nu toe
# zonder SMOTE omdat we merken dat SMOTE niet bijdraagt aan een betere score
# we gebruiken wel de parameter class_weight='balanced' om de onbalans in de target value te compenseren

# duurt enkele minuten

rfc = RandomForestClassifier()
abc = AdaBoostClassifier()
gbc = GradientBoostingClassifier()
etc = ExtraTreesClassifier()
svc = SVC()
lgr = LogisticRegression(class_weight='balanced')

models_cl = [rfc, abc, gbc, etc, lgr, svc]
models_cl = [lgr, lgr, lgr]

names_cl = ["Random Forest", "Ada Boost", "Gradient Boosting", "Extra Trees",
        "Logistic Regression", "Support Vector Machine"]
names_cl = ["Logistic Regression", "Support Vector Machine","logistic regression"]

def training_classification(X_train, X_test, y_train, y_test, preprocessor):    
    scores, scores_dict, reports, cms = [], dict(), dict(), dict()
    for i, j in zip(models_cl, names_cl):
        pipeline = Pipeline([
            ('preprocessor', preprocessor),                  
            ('regressor', i)            
            ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        acc_score = accuracy_score(y_test, y_pred)*100
        scores += [acc_score]
        scores_dict[j] = [acc_score]    
        reports[j] = classification_report(y_test, y_pred)
        cms[j] = confusion_matrix(y_test, y_pred)
        log_info(f"\n\nModel : {j}")
        log_info("\n")
        log_info(f"Accuracy scores: {acc_score}")
        log_info(f"Confusion matrix: {cms[j]}") 
        log_info(f"Classification report: {reports[j]}") 
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=pipeline.classes_)
        disp.plot()
        fig_titel = f"ConfusionMatrixDisplay voor model {i}"
        save_fig(fig_titel)

    fig, axes = plt.subplots()
    df_results = pd.DataFrame({"score": scores}, index=names_cl)
    df_results = df_results.sort_values("score", ascending=False)

    # Een bar plot maken
    plt.figure(figsize=(10, 5))  # Grootte van de figuur instellen
    plt.bar(names_cl, scores, color='blue')  # Staafdiagram tekenen

    # Labels en titel toevoegen
    plt.xlabel('Model')  # Label voor de x-as
    plt.ylabel('Accuracy score')  # Label voor de y-as
    plt.title('Vergelijking van modellen voor classificatie')  # Titel van de plot

    # De plot tonen
    plt.show()

    index = 0
    for row in range(2):
        fig, axes = plt.subplots(ncols=3, figsize=(15, 6))        
        for i in range(3):
            sns.heatmap(cms[df_results.index[index]], annot=True, fmt='d', ax=axes[i])
            axes[i].set_title("{}: {}%".format(df_results.index[index], df_results.iloc[index, 0]))
            index += 1
        plt.tight_layout()
        plt.show()
   
    return scores_dict, reports, cms

In [ ]:
df = df_raw.copy()

# Separate features (X) and target variable (y)
X = df.drop(columns=['Heart_Attack_Risk'])
y = df['Heart_Attack_Risk']

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

scores_dict, reports_dict, cms_dict = training_classification(X_train, X_test, y_train, y_test, preprocessor)

#results_string = tabulate(results, headers='keys', tablefmt='psql')
for models in scores_dict.keys():
    log_info("\n")
    log_info(f"Resultaten voor model {models}")
    log_info("**********************************************\n")
    log_info(f"Score: {scores_dict[models]}")
    log_info(f"Classification report: {reports_dict[models]}")
    log_info(f"Confusion matrix: {cms_dict[models]}")
    log_info("\n\n")

df = pd.DataFrame(scores_dict)
print(df)
df.plot(kind='bar', figsize=(10, 5))
save_fig("Vergelijking van modellen voor classificatie zonder Hyperparameter tuning")


En nu gaan we de hyperparameters van de diverse modellen tunen  met GridSearchCV

In [ ]:
def execute_classification_models_with_pipeline_hyperparameter_tuning(X_train, X_test, y_train, y_test, preprocessor):
    # Hyperparameters sets voor verschillende modellen

    models = {
        'Logistic Regression': LogisticRegression(),
        # 'Gradient Boosting': GradientBoostingClassifier()
    }

    param_grid = {
        'Logistic Regression': {
            'classifier__C': [0.01, 0.1, 1.0],  # Regularisatieparameter
            'classifier__solver': ['liblinear', 'lbfgs']  # Solvers die verschillende penalties ondersteunen
        },
        'Gradient Boosting': {
            'classifier__n_estimators': [100, 200],
            'classifier__learning_rate': [0.1, 0.05, 0.01]
        }
    }
    
    results = {}
    for model_name, model in models.items():
        pipeline = Pipeline([
            ('preprocessor', preprocessor),           
            ('classifier', model)
        ])     

        grid_search = GridSearchCV(pipeline, param_grid[model_name], scoring="f1_weighted", cv=5, refit='f1')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)

        results[model_name] = grid_search.best_estimator_
        best_score = grid_search.best_score_

        test_score = best_model.score(X_test, y_test)
        print("Score op de testset:", test_score)

        y_pred = best_model.predict(X_test)
        conf_matrix = confusion_matrix(y_test, y_pred)
        class_report = classification_report(y_test, y_pred)   

        print(test_score)

        print(conf_matrix)
        print(class_report)

        results[model_name] = {'test_score': test_score, 'conf_matrix': conf_matrix, 'class_report': class_report}        
    return results

In [ ]:
# Define categorical and numerical column names
df_col = X.columns  
df_num_col = ['Age', 'BMI', 'Cholesterol_Level', 'Resting_BP', 'Heart_Rate', 'Max_Heart_Rate_Achieved']
df_cat_col = list(set(df_col) - set(df_num_col))

num_pipeline = Pipeline([   
    ("standardize_numerical", MinMaxScaler(feature_range=(-1, 1)))
    # ("standardize_numerical", StandardScaler())
])

cat_pipeline = Pipeline(steps=[   
    ('encode_categorical', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer([
    ("numerical", num_pipeline, df_num_col),
    ("categorical", cat_pipeline, df_cat_col)],
     remainder='passthrough')

# Create the pipeline with preprocessing and logistic regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),       
    ('regressor', LogisticRegression(class_weight='balanced'))
])

df = df_raw.copy() 
X = df.drop('Heart_Attack_Risk', axis=1)
y = df['Heart_Attack_Risk']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
results = execute_classification_models_with_pipeline_hyperparameter_tuning(X_train, X_test, y_train, y_test, preprocessor)

df_results = pd.DataFrame(results, columns=['model', 'accuracy', 'report', 'cms'])
resultaat_string = tabulate(df_results, headers='keys', tablefmt='psql')

log_info("\n")
log_info(f"Resultaten voor model met hyperparameter tuning")
log_info("**********************************************\n")
log_info(f"voor models : {results.keys()}\n\n")

for keys, items in results.items():
    log_info(f"Resultaat voor model {keys}")     
    for k, item in items.items():
        log_info(f"Parameter: {k}")     
        log_info(f"{item}")    
    log_info("\n\n")


# haal uit de resultsdict alle waarden van de accuracy en plot deze per model
# en maak een confusion matrix voor elk model
model_names = list(results.keys())
accuracies = [results[model]['test_score'] for model in model_names]

# Plot the accuracies
plt.figure(figsize=(10, 5))
plt.bar(model_names, accuracies, color='blue')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Accuracy of Different Models with Hyperparameter Tuning')
plt.xticks(rotation=45)
save_fig("Model_accuracies_with_hyperparameter_tuning")
plt.show()

